# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/citiesEhlke.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Adamstown,-25.0660,-130.1015,73.58,80,100,21.39,1019,PN,1733951261
1,1,Minbu,20.1833,94.8833,64.58,76,11,4.09,1010,MM,1733951262
2,2,Mar del Plata,-38.0023,-57.5575,69.94,83,98,4.00,1011,AR,1733951210
3,3,Edinburgh of the Seven Seas,-37.0676,-12.3116,57.76,72,99,13.35,1024,SH,1733951264
4,4,Bethel,41.3712,-73.4140,60.04,90,100,7.00,995,US,1733950988


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
city_data_df2 = city_data_df.dropna()
mask = (city_data_df2["Max Temp"] >= min_temp) & (city_data_df2["Max Temp"] <= max_temp) & (city_data_df2["Wind Speed"] < max_wind)
city_data_df2 = city_data_df2.loc[mask].reset_index(drop=True)

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,13,Rémire-Montjoly,4.9167,-52.2667,78.84,94,75,3.44,1009,GF,1733951275
1,16,Puerto Princesa City,9.7392,118.7353,79.30,88,100,9.17,1007,PH,1733951278
2,23,Lompoc,34.6391,-120.4579,71.69,38,0,6.91,1018,US,1733951184
3,26,Kalmunai,7.4167,81.8167,77.95,91,95,7.52,1006,LK,1733951291
4,35,Mananara,-16.1667,49.7667,73.99,90,38,4.27,1014,MG,1733951300
...,...,...,...,...,...,...,...,...,...,...,...
84,539,Dampit,-8.2116,112.7493,72.34,95,100,3.06,1007,ID,1733951885
85,543,Boende,-0.2167,20.8667,74.73,83,92,2.89,1010,CD,1733951890
86,547,Yanam,16.7333,82.2167,75.29,78,88,6.40,1011,IN,1733951895
87,552,Hilo,19.7297,-155.0900,79.11,79,40,9.22,1021,US,1733951608


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
hotel_df = city_data_df2[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Rémire-Montjoly,GF,4.9167,-52.2667,94,
1,Puerto Princesa City,PH,9.7392,118.7353,88,
2,Lompoc,US,34.6391,-120.4579,38,
3,Kalmunai,LK,7.4167,81.8167,91,
4,Mananara,MG,-16.1667,49.7667,90,
...,...,...,...,...,...,...
84,Dampit,ID,-8.2116,112.7493,95,
85,Boende,CD,-0.2167,20.8667,83,
86,Yanam,IN,16.7333,82.2167,78,
87,Hilo,US,19.7297,-155.0900,79,


In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Rémire-Montjoly - nearest hotel: Complexe Belova
Puerto Princesa City - nearest hotel: Circon Businessman's Inn
Lompoc - nearest hotel: Red Roof Inn Lompoc
Kalmunai - nearest hotel: Saji new food
Mananara - nearest hotel: Chez Roger
Port Elizabeth - nearest hotel: Waterford Hotel
Medina - nearest hotel: The Oberoi Madina
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Ixtapa - nearest hotel: Ma. Cristina
Hawaiian Paradise Park - nearest hotel: No hotel found
Katabu - nearest hotel: No hotel found
Nova Viçosa - nearest hotel: Pousada Pontal da Barra
São Tomé - nearest hotel: Hotel Baia
Conduaga - nearest hotel: No hotel found
Booué - nearest hotel: La splendeur de l'équateur
Grand Gaube - nearest hotel: Hotel Lux- Grand Gaube
Lodja - nearest hotel: No hotel found
Kourou - nearest hotel: SCI Horizon
Yoko - nearest hotel: No hotel found
Pariaman - nearest hotel: HOTEL NAN TONGGA
Gamba - nearest hotel: No hotel found
Biltine - nearest hotel: فندق ثل

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Rémire-Montjoly,GF,4.9167,-52.2667,94,Complexe Belova
1,Puerto Princesa City,PH,9.7392,118.7353,88,Circon Businessman's Inn
2,Lompoc,US,34.6391,-120.4579,38,Red Roof Inn Lompoc
3,Kalmunai,LK,7.4167,81.8167,91,Saji new food
4,Mananara,MG,-16.1667,49.7667,90,Chez Roger
...,...,...,...,...,...,...
84,Dampit,ID,-8.2116,112.7493,95,No hotel found
85,Boende,CD,-0.2167,20.8667,83,Hotel Reference
86,Yanam,IN,16.7333,82.2167,78,Hôtel Sarvainn
87,Hilo,US,19.7297,-155.0900,79,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["Hotel Name", "Country"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE